In [9]:
%%bash
NAMESPACE=`cat /var/run/secrets/kubernetes.io/serviceaccount/namespace`
cat <<EOF | kubectl apply -n $NAMESPACE -f -
apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "huggingface"
  annotations:
    sidecar.istio.io/inject: "false"
    # https://github.com/istio/istio/issues/23802
    traffic.sidecar.istio.io/excludeOutboundIPRanges: 0.0.0.0/0
spec:
  predictor:
    containers:
      - name: huggingface
        image: mtr.devops.telekom.de/ai/huggingface:latest
        resources:
          limits:
            cpu: '1'
            memory: 20Gi
          requests:
            cpu: '1'
            memory: 20Gi        
EOF

I0614 18:22:26.793436     302 request.go:668] Waited for 1.175871858s due to client-side throttling, not priority and fairness, request: GET:https://172.30.0.1:443/apis/template.openshift.io/v1?timeout=32s
I0614 18:22:36.993238     302 request.go:668] Waited for 11.3753377s due to client-side throttling, not priority and fairness, request: GET:https://172.30.0.1:443/apis/argoproj.io/v1alpha1?timeout=32s


inferenceservice.serving.kserve.io/huggingface created


In [1]:
%%bash
NAMESPACE=`cat /var/run/secrets/kubernetes.io/serviceaccount/namespace`
for i in {1..12}
do
   STATE=`kubectl -n $NAMESPACE get inferenceservice huggingface -o=jsonpath='{.status.components.predictor.latestReadyRevision}'`
   echo -e "$STATE"
   if [[ "$STATE" == "" ]]; then echo -e "not ready"; fi
   if [[ "$STATE" != "" ]]; then echo "$STATE"; break; fi
   sleep 10s
done

I0608 08:30:37.709831     807 request.go:668] Waited for 1.173352545s due to client-side throttling, not priority and fairness, request: GET:https://172.30.0.1:443/apis/crunchydata.com/v1?timeout=32s
I0608 08:30:47.710221     807 request.go:668] Waited for 11.173284038s due to client-side throttling, not priority and fairness, request: GET:https://172.30.0.1:443/apis/psmdb.percona.com/v1-3-0?timeout=32s


huggingface-predictor-default-00001
huggingface-predictor-default-00001


In [2]:
%%writefile huggingface-input.json
{    "instances":[  
    	{
    		"text": "Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between TensorFlow 2.0 and PyTorch.",
          "questions":[
		    "How many pretrained models are available in Transformers?",
		    "What does Transformers provide?",
		    "Transformers provides interoperability between which frameworks?"
          ]
       }
    ]
}

Writing huggingface-input.json


In [1]:
%%bash
NAMESPACE=`cat /var/run/secrets/kubernetes.io/serviceaccount/namespace`
INFERENCESERVICE=huggingface
SERVICE_HOSTNAME=$INFERENCESERVICE.$NAMESPACE.svc.cluster.local
GATEWAY='http://knative-local-gateway.istio-system.svc.cluster.local'
curl -H "Host: ${SERVICE_HOSTNAME}" \
    $GATEWAY/v1/models/$INFERENCESERVICE:predict -d @huggingface-input.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   890  100   260  100   630     27     67  0:00:09  0:00:09 --:--:--    62


{"predictions": {"How many pretrained models are available in Transformers?": "over 32 +", "What does Transformers provide?": "general - purpose architectures", "Transformers provides interoperability between which frameworks?": "tensorflow 2. 0 and pytorch"}}